# Running a Simulation

Now that we have tuned the simulation’s parameter it is time to run it. Because this is another notebook we copy the first cell of the [Pre Simulation Testing notebook](./Pre_Simulation_Testing.ipynb) and then run the simulation with only three lines of code.

The YAML input file can be found at [input_file](https://raw.githubusercontent.com/murillo-group/sarkas/master/docs/documentation/Tutorial_NB/input_files/yukawa_mks_p3m.yaml) and this notebook at [notebook](https://raw.githubusercontent.com/murillo-group/sarkas/master/docs/documentation/Tutorial_NB/Simulation_Docs.ipynb)



In [1]:
# Import the usual libraries
import os

# Import sarkas
from sarkas.processes import Simulation

# Create the file path to the YAML input file
input_file_name = os.path.join('input_files', 'yukawa_mks_p3m.yaml')

Then we run our simulation with only three lines of code

In [2]:
sim = Simulation(input_file_name)
sim.setup(read_yaml=True)
sim.run()







 __            _             
/ _\ __ _ _ __| | ____ _ ___ 
\ \ / _` | '__| |/ / _` / __|
_\ \ (_| | |  |   < (_| \__ \
\__/\__,_|_|  |_|\_\__,_|___/
                             


An open-source pure-python molecular dynamics suite for non-ideal plasmas.




********************************************************************************
                                   Simulation                                   
********************************************************************************

Job ID: yocp
Job directory: SarkasSimulations/yocp_pppm
Simulation directory: 
SarkasSimulations/yocp_pppm/Simulation

Equilibration dumps directory: 
SarkasSimulations/yocp_pppm/Simulation/Equilibration/dumps
Production dumps directory: 
SarkasSimulations/yocp_pppm/Simulation/Production/dumps

Equilibration H5MD file: 
SarkasSimulations/yocp_pppm/Simulation/Equilibration/dumps/yocp_data.h5md
Production H5MD file: 
SarkasSimulations/yocp_pppm/Simulation/Production/dumps/yocp_data.h5md

  0%|          | 0/7500 [00:00<?, ?it/s]


Equilibration Time: 0 hrs 16 min 26 sec


------------------------------Production------------------------------ 



  0%|          | 0/7500 [00:00<?, ?it/s]


Production Time: 0 hrs 16 min 21 sec

Total Time: 0 hrs 32 min 48 sec


========================= Filesize Estimates =========================

Equilibration:
	H5MD filesize: 0 GB 519 MB 935 KB 416 bytes

Production:
	H5MD filesize: 0 GB 519 MB 935 KB 416 bytes

Total occupied space: 1 GB 15 MB 846 KB 832 bytes


The verbose output of the simulation gives the same information as `PreProcess`. The first few lines tell us where the simulations' data is stored. Notice that these are diffrent than what was printed by `PreProcessing`. 

>**_NOTE:_**  This simulation was run on a 2019 Dell XPS 8930 with Intel Core i7-8700K @ 3.70Ghz and 48GB of RAM running Ubuntu 18.04. Actual run times will be different on your computer.

## H5MD
The simulation data is saved in a H5MD file. Each dump is a [.npz](https://numpy.org/doc/stable/reference/generated/numpy.savez.html) file containing the following information for each of phase:

    Equilibration:
     id = particles id's numbers, 
     names = species name of each particle,
     pos = Particles positions N x 3 array,
     vel = Particles velocities N x 3 array,
     acc = Particles acceleration N x 3 array,
     virial = Virial matrix N x 3 x 3 array,
     time = Time in seconds

    Production:
     id = particles id's numbers, 
     names = species name of each particle,
     pos = Particles positions N x 3 array,
     vel = Particles velocities N x 3 array,
     acc = Particles acceleration N x 3 array,
     cntr = Number of times each particle has been folded back into the simulation box, N x 3 array,
     rdf_hist = Instantaneous Histogram of the Radial Distribution Function,
     virial = Virial matrix N x 3 x 3 array,
     time = Time in seconds

This information is saved by the [.dump()](../../api/util_subpckg/InputOutput_methods/sarkas.utilities.io.InputOutput.dump.rst#sarkas.utilities.io.InputOutput.dump) method of the [InputOutput](../../api/util_subpckg/sarkas.utilities.io.InputOutput.rst).

The same method saves Energy and Temperature information in the Thermodynamics files also. This can be accessed by `sim.io.eq_energy_filename` and `sim.io.prod_energy_filename`. The information saved is the following:

    Time, Total Energy, Total Kinetic Energy, Potential Energy, Total Temperature

In addition, in the case of multispecies plasmas, the kinetic energy and the temperature of each species will be saved.

Once the simulation is complete it is time to analyze the data. Follow this [link](./Post_Processing_Docs.rst).

### Explanation of H5MD Files

H5MD (HDF5 for Molecular Dynamics) files are a standardized format for storing molecular dynamics data in a structured, efficient, and portable manner. The H5MD format is based on the HDF5 file format, which provides a flexible and efficient way to store large amounts of data.

The H5MD format is described in detail in the paper by Pierre de Buyl et al., titled "H5MD: A structured, efficient, and portable file format for molecular data," [link to paper](https://doi.org/10.1016/j.cpc.2014.01.018).

### File Structure of H5MD Files

The structure of an H5MD file is organized into several groups, each containing datasets and attributes that store different types of MD data. The main groups include:

1. **`h5md` Group**: This group contains metadata about the H5MD format version and general information about the simulation.

2. **`particles` Group**: This group is used to store data related to particles, such as positions, velocities, accelerations, and other particles properties. The positions, velocities, and accelerations subgroups hold time-dependent datasets and static properties of the particles. For example:

        Positions: Stored in a subgroup called position, which contains three datasets:
            value Dataset: Stores the x, y, and z coordinates of particles.
                Shape: (timesteps, num_particles, 3)
                Data Type: float64
            time Dataset: Stores the timestamps for each recorded position.
                Shape: (timesteps,)
                Data Type: float64
            step Dataset: Stores the simulation steps corresponding to each recorded position.
                Shape: (timesteps,)
                Data Type: int64

3. **`box` Group**: This group is contained in `particles` and it contains information about the simulation box, such as its dimensions and periodicity.

4. **`observables` Group**: This group contains the thermodynamic quantities of each species and observables that are general, such as `rdf_hist`, which is the histogram used to calculate the pair distribution function obtained while running the simulations. For example:

        [species_name]/temperature: Stored in within a subgroup with the species name.
            value Dataset: Stores the temperature values of the system.
                Shape: (timesteps,)
                Data Type: float64
            time Dataset: Stores the timestamps for each recorded temperature.
                Shape: (timesteps,)
                Data Type: float64
            step Dataset: Stores the simulation steps corresponding to each recorded temperature.
                Shape: (timesteps,)
                Data Type: int64

   
        rdf_hist: Stored in a subgroup called rdf_hist, which contains three datasets:
            value Dataset: Stores the histogram values.
                Shape: (timesteps, num_species, num_species, num_bins)
                Data Type: float64
            time Dataset: Stores the timestamps for each recorded histogram.
                Shape: (timesteps,)
                Data Type: float64
            step Dataset: Stores the simulation steps corresponding to each recorded histogram.
                Shape: (timesteps,)
                Data Type: int64

The following code provides an example of how to access and interact with datasets stored in an H5MD file using the `h5py` library. This code is structured to read specific datasets, list available datasets within a group, and print relevant data along with their attributes.


In [3]:
import h5py

# Function to list all datasets in a given group
def list_datasets(group):
    """
    List all datasets in a given HDF5 group.

    Parameters
    ----------
    group : h5py.Group
        The HDF5 group to list datasets from.

    Returns
    -------
    list of str
        A list of dataset names within the specified group.
    """
    datasets = []  # Initialize an empty list to store dataset names
    for name, item in group.items():  # Iterate over all items in the group
        if isinstance(item, h5py.Dataset):  # Check if the item is a dataset
            datasets.append(name)  # Add the dataset name to the list
    return datasets  # Return the list of dataset names

# Open the H5MD file in read-only mode
with h5py.File(sim.io.h5md_filenames_tree["simulation"]["production"], 'r') as file:

    print(f"The groups in the H5MD file are:\n{file.keys()}")
    
    # Open the positions subgroup
    particles_group = file["particles"]
    
    # List all datasets in the 'temperature' subgroup under 'observables/H'
    datasets = list_datasets(particles_group)
    print("\nThe particles group has these datasets:")
    for dataset in datasets:  # Iterate over the dataset names
        print(f"{dataset} \tshape: {particles_group[dataset].shape}") # Print each dataset name and shape

    # Access the 50th entry in the 'particles/pos' dataset and print the first particles position
    print(f"\nThe position of the first particle at the 500th timestep is: {particles_group['pos'][50,0]}") 
    # 50 because we save data every 10 timesteps

    
    # Access the 'observables' group in the HDF5 file
    observables_group = file["observables"]
    print(f"\nThe subgroups of observables are:\n{observables_group.keys()}")  # Print all keys (subgroups) in the 'observables' group

    # Access the thermodynamic properties stored under the "observables/H" group
    species_thermodynamics = file["observables/H"]
    print(f"\nThe stored properties (subgroups) of H are:\n{species_thermodynamics.keys()}")  # List all keys under the 'observables/H' group to see what properties are stored

    # List all datasets in the 'temperature' subgroup under 'observables/H'
    datasets = list_datasets(species_thermodynamics["temperature"])
    print("\nEach of the has three datasets:")
    for dataset in datasets:  # Iterate over the dataset names
        print(dataset)  # Print each dataset name

    # Access the 'temperature' data from the 'observables/H' group
    temp = species_thermodynamics["temperature"]
    print(f"\nThe temperature has shape: {temp['value'].shape}")  # Print the shape of the 'value' dataset in 'temperature'

    # Print the temperature value at the 500th timestep (corresponding to index 50 in the saved data)
    # Also, print the units of the temperature, which are stored as an attribute
    print(f"The temperature at the 50th dump is: {temp['value'][50]:.2f} {temp.attrs['units']}")  # Access and print the temperature value and units


The groups in the H5MD file are:
<KeysViewHDF5 ['h5md', 'observables', 'particles']>

The particles group has these datasets:
acc 	shape: (751, 10000, 3)
charges 	shape: (10000,)
masses 	shape: (10000,)
names 	shape: (10000,)
pos 	shape: (751, 10000, 3)
species 	shape: (10000,)
step 	shape: (751,)
time 	shape: (751,)
vel 	shape: (751, 10000, 3)

The position of the first particle at the 500th timestep is: [2.55152125e-10 3.56385370e-10 3.11884623e-10]

The subgroups of observables are:
<KeysViewHDF5 ['H', 'rdf_hist']>

The stored properties (subgroups) of H are:
<KeysViewHDF5 ['kinetic_energy', 'potential_energy', 'temperature', 'total_energy']>

Each of the has three datasets:
step
time
value

The temperature has shape: (751,)
The temperature at the 50th dump is: 5784.78 [K]
